In [1]:
import pandas as pd

In [2]:
hotels4 = pd.read_csv('hotels4.csv')

In [5]:
hotels=pd.read_csv('Hotel Location.csv',encoding='latin1')


In [9]:
hotels.sample(2)

,Name,Address1,Address2,City,StateProvince,PostalCode,Country,Latitude,Longitude,AirportCode,PropertyCurrency,StarRating,Location
93067,Microtel Inn by Wyndham Champaign,1615 Rion Drive,NaN,Champaign,IL,61822,US,40.15968,-88.27357,CMI,USD,2.0,In the city center
4003,Americas Best Value Inn,501 Virginia Rd,Us-17 Bypass & Rt.32 Exit#227,Edenton,NC,27932,US,36.07952,-76.61981,PGV,USD,2.0,Near Cupola House Association


In [10]:
hotels4.head(2)

,scraped_url,agency_slug,country,title,description,inclusions,hotel_name,hotel_mapped,domain,hotels_new,value
0,https://www.flightcentre.co.za/product/14197346,flightcentre,maldives,Adaaran Select Meedhupparu,\nYour Maldives Holiday Package includes:\nRet...,\nYour Maldives Holiday Package includes:\n\nR...,NaN,NaN,flightcentre,Adaaran Select Meedhupparu,NaN
1,https://www.flightcentre.co.za/product/5415249,flightcentre,south africa,Family Fun on the Wild Coast,\nYour Eastern Cape Holiday Package includes:\...,\nYour Eastern Cape Holiday Package includes:\...,NaN,NaN,flightcentre,Wild Coast Sun,NaN


In [13]:
hotels5 = hotels4.drop(['agency_slug','country','title','description','inclusions','hotel_name','hotel_mapped','domain','value','scraped_url'], axis=1)


In [15]:
hotels5.sample(5)

,hotels_new
4575,Cruise Japan with Holland America
5831,Riu Palace Zanzibar
162,Cathedral Peak Hotel  Drakensberg
3961,12 Night South Africa & Namibia Cruise
3507,veranda pointe aux biches


In [28]:
hotels6 = hotels5.dropna()

In [16]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\user\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [17]:
names = list(hotels['Name'])

In [20]:
process.extractOne('veranda pointe aux biches', names)

('Veranda Pointe Aux Biches Hotel', 95)

In [22]:
import re
!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

def ngrams(string, n=3):
    string = str(string)
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

ERROR: Invalid requirement: '#'


In [23]:
import numpy as np
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn #uncomment to install
import sparse_dot_topn.sparse_dot_topn as ct


def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

ERROR: Invalid requirement: '#uncomment'


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

org_names = hotels['Name'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(org_names)

In [29]:
org_names1 = hotels6['hotels_new']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix1 = vectorizer.fit_transform(org_names1)

In [ ]:
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix1, 10, 0.85)
t = time.time()-t1
print("SELFTIMED:", t)

In [25]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [26]:
matches_df = get_matches_df(matches, org_names, top=1000)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
matches_df.sample(20)

NameError: name 'matches' is not defined